In [ ]:
%matplotlib inline



# Compare image and coordinate based meta-analyses

Run IBMAs and CBMAs on a toy dataset, then compare the results qualitatively.

Collection of NIDM-Results packs downloaded from Neurovault collection 1425,
uploaded by Dr. Camille Maumet.


In [ ]:
import os

import pandas as pd
from nilearn.plotting import plot_stat_map

from nimare.dataset import Dataset
from nimare.extract import download_nidm_pain
from nimare.meta.cbma import ALE
from nimare.meta.ibma import DerSimonianLaird
from nimare.transforms import ImagesToCoordinates, ImageTransformer
from nimare.utils import get_resource_path

## Download data



In [ ]:
dset_dir = download_nidm_pain()

## Load Dataset



In [ ]:
dset_file = os.path.join(get_resource_path(), "nidm_pain_dset.json")
dset = Dataset(dset_file)
dset.update_path(dset_dir)

# Calculate missing statistical images from the available stats.
xformer = ImageTransformer(target=["z", "varcope"])
dset = xformer.transform(dset)

# create coordinates from statistical maps
coord_gen = ImagesToCoordinates(merge_strategy="replace")
dset = coord_gen.transform(dset)

## ALE (CBMA)



In [ ]:
meta_cbma = ALE()
meta_cbma.fit(dset)
plot_stat_map(
    meta_cbma.results.get_map("z"), cut_coords=[0, 0, -8], draw_cross=False, cmap="RdBu_r"
)

## DerSimonian-Laird (IBMA)
We must resample the image data to the same MNI template as the Dataset.



In [ ]:
meta_ibma = DerSimonianLaird(resample=True)
meta_ibma.fit(dset)
plot_stat_map(
    meta_ibma.results.get_map("z"), cut_coords=[0, 0, -8], draw_cross=False, cmap="RdBu_r"
)

## Compare CBMA and IBMA Z-maps



In [ ]:
stat_df = pd.DataFrame(
    {
        "CBMA": meta_cbma.results.get_map("z", return_type="array"),
        "IBMA": meta_ibma.results.get_map("z", return_type="array").squeeze(),
    }
)
print(stat_df.corr())